# Population Data from CSV

This notebooks reads sample population data from `data/atlantis.csv` and plots it using Matplotlib. Edit `data/atlantis.csv` and re-run this cell to see how the plots change!

In [13]:
pip install "ray[rllib]" tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install swig


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install gymnasium[box2d]


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip install wandb


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [17]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import random
import wandb

#мы заменили float и np.array
class GoLeftEnv(gym.Env):
  """
  Custom Environment that follows gym interface.
  This is a simple env where the agent must learn to go always left.
  """
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {'render.modes': ['console']}
  # Define constants for clearer code
  FIRST = 0
  SECOND = 1
  THIRD = 2
  FORTH = 3
  FIFTH= 4
  SIXTH = 5
  SEVENTH = 6

  def __init__(self, grid_size=10, render_mode="console"):
    super(GoLeftEnv, self).__init__()
    self.render_mode = render_mode

    self.fields = np.array([[0, 0, 7, 7,], [1, 0, 0, 0,], [2, 0, 0, 0,], [3, 0, 0, 0,], [4, 1, 0, 0,], [5, 0, 0, 0], [6, 0, 0, 0], [7, 0, 0, 0], [8, 1, 0, 0], [9, 0, 0, 0], [10, 0, 0, 0], [11, 0, 0, 0], [12, 0, 0, 0],
                   [13, 0, 0, 0], [14, 1, 0, 0], [15, 0, 0, 0], [0, 0, 0, 0]])
    

    self.fields = self.fields.flatten('C')

    self.fields = [x/16 for x in self.fields]

    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions, we have two: left and right
    n_actions = 7
    self.action_space = spaces.Discrete(n_actions)
    # The observation will be the coordinate of the agent
    # this can be described both by Discrete and Box space
    self.observation_space = spaces.Box(low=0, high=1,
                                        shape=(68,), dtype=np.float32)
  #попробовать переделать всё в np.array

  def reset(self, seed=None, options=None):
    """
    Important: the observation must be a numpy array
    :return: (np.array)
    """
    super().reset(seed=seed, options=options)


    # Initialize the agent at the right of the grid
    self.fields = np.array([[0, 0, 7, 7,], [1, 0, 0, 0,], [2, 0, 0, 0,], [3, 0, 0, 0,], [4, 1, 0, 0,], [5, 0, 0, 0], [6, 0, 0, 0], [7, 0, 0, 0], [8, 1, 0, 0], [9, 0, 0, 0], [10, 0, 0, 0], [11, 0, 0, 0], [12, 0, 0, 0],
                   [13, 0, 0, 0], [14, 1, 0, 0], [15, 0, 0, 0], [0, 0, 0, 0]])


    self.fields = self.fields.flatten('C')

    self.fields = [x/16 for x in self.fields]

    # here we convert to float32 to make it more general (in case we want to use continuous actions)
    return np.array(self.fields).astype(np.float32), {}

  def step(self, action):
     
     self.fields = [x if x == 0 else int(x*16) for x in self.fields]

     self.fields = [int(a) for a in self.fields]

     #print(self.fields)

     self.fields = np.array(self.fields)

     self.fields = self.fields.reshape(17, 4)

     #print(self.fields)

     def first():
            numberforcycles = 15

            balcycle1 = 0

            roll = self.fields[16][0]

            lst = [0, 1, 2, 3, 4]
            weights = [6.25, 25, 37.5, 25, 6.25]
            nextroll = random.choices(lst, weights=weights, k=1)
            nextroll = nextroll[0]
            self.fields[16][0] = nextroll
            #print("Выпала" , roll)
            arrplayer1 = []
            while balcycle1 < numberforcycles:
              ballnow = self.fields[balcycle1][2]
              ballindex = self.fields[balcycle1][0]
          #расположение фишек игрока 1
              if ballnow != 0:
               arrplayer1.append(ballindex)
              balcycle1 += 1
            if self.fields[8][3] == 1:
                arrplayer1.append(8)
            #print(arrplayer1)

            balcycle1 = 0
            arrchoice1 = []
            while balcycle1 < numberforcycles:
              ballnow = self.fields[balcycle1][2]
              ballindex = self.fields[balcycle1][0]
              if ballnow != 0:
                ballfuture = ballindex + roll
                if ballfuture <= numberforcycles:
                 if ballfuture not in arrplayer1:
                  arrchoice1.append(ballfuture)
              # print("Выпала" , self.roll)
                  #print("текущая позиция" , ballindex)
                  #print("возможная позиция" , ballfuture)
              balcycle1 += 1



            if arrchoice1:

              #случайный бот:
              #playeronemove = random.choice(arrchoice1)

              #Очень жадный бот:
              playeronemove = len(arrchoice1)
              playeronemove = arrchoice1[playeronemove-1]

              #Жадный бот:
              #playeronemove = 33
              #for arrchoice in arrchoice1:
              #  if self.fields[arrchoice][1] == 1 or (self.fields[arrchoice][3] == 1 and self.fields[arrchoice][0] >= 5 and self.fields[arrchoice][0] <= 12):
              #    playeronemove = self.fields[arrchoice][0]
              #if playeronemove == 33:
              #  playeronemove = len(arrchoice1)
              #  playeronemove = arrchoice1[playeronemove-1]

              #playeronemove = random.choice(arrchoice1)
              playeronepos = playeronemove - roll
              #print(playeronepos)
              self.fields[playeronepos][2] = self.fields[playeronepos][2] - 1
              self.fields[playeronemove][2] = self.fields[playeronemove][2] + 1
              if self.fields[playeronemove][3] == 1 and playeronemove >= 5 and playeronemove <= 12:
                self.fields[playeronemove][3] = 0
                self.fields[0][3] = self.fields[0][3] + 1
                #print("мы забрали шашку на" , self.fields[playeronemove][0])
              #print("боту выпал", roll)
              #print("бот ходит", playeronemove)
              return playeronemove

     def second():
            numberforcycles = 15

            agentroll = self.fields[16][1]


            lst = [0, 1, 2, 3, 4]
            weights = [6.25, 25, 37.5, 25, 6.25]
            nextagentroll = random.choices(lst, weights=weights, k=1)
            nextagentroll = nextagentroll[0]
            self.fields[16][1] = nextagentroll
            #print("Агенту Выпал" , agentroll)

            agentcycle = 0
            arrplayeragent = []

            while agentcycle < numberforcycles:
              agentballnow = self.fields[agentcycle][3]
              agentballindex = self.fields[agentcycle][0]
          #расположение фишек игрока 2
              if agentballnow != 0:
               arrplayeragent.append(agentballindex)
              agentcycle += 1
            if self.fields[8][2] == 1:
                arrplayeragent.append(8)

            #print("фишки агента", arrplayeragent)

            agentcycle = 0
            arrplayeragent2 = []

            while agentcycle < numberforcycles:
              agentballnow = self.fields[agentcycle][3]
              agentballindex = self.fields[agentcycle][0]
          #будущее фишек игрока 2
              if agentballnow != 0:
               agentballfuture = agentballindex + agentroll
               if agentballfuture <= numberforcycles:
                if agentballfuture not in arrplayeragent:
                  arrplayeragent2.append(agentballfuture)
              agentcycle += 1

            #print(arrplayeragent2)



            if arrplayeragent2:
              playeragentmove = len(arrplayeragent2)
              playeragentmove = arrplayeragent2[playeragentmove-1]

              if action == self.FIRST:
                if len(arrplayeragent2) > 0:
                 playeragentmove = arrplayeragent2[0]
                else:
                 playeragentmove = playeragentmove
              elif action == self.SECOND:
               if len(arrplayeragent2) > 1:
                playeragentmove = arrplayeragent2[1]
               else:
                playeragentmove = playeragentmove
              elif action == self.THIRD:
               if len(arrplayeragent2) > 2:
                playeragentmove = arrplayeragent2[2]
               else:
                playeragentmove = playeragentmove
              elif action == self.FORTH:
               if len(arrplayeragent2) > 3:
                playeragentmove = arrplayeragent2[3]
               else:
                playeragentmove = playeragentmove
              elif action == self.FIFTH:
               if len(arrplayeragent2) > 4:
                playeragentmove = arrplayeragent2[4]
               else:
                playeragentmove = playeragentmove
              elif action == self.SIXTH:
               if len(arrplayeragent2) > 5:
                playeragentmove = arrplayeragent2[5]
               else:
                playeragentmove = playeragentmove
              elif action == self.SEVENTH:
               if len(arrplayeragent2) > 6:
                playeragentmove = arrplayeragent2[6]
               else:
                playeragentmove = playeragentmove
              else:
                raise ValueError("Received invalid action={} which is not part of the action space".format(action))



              #print("Агент делает выбор", playeragentmove)

              #playeragentmove = random.choice(arrplayeragent)
              playeragentpos = playeragentmove - agentroll
              self.fields[playeragentpos][3] = self.fields[playeragentpos][3] - 1
              self.fields[playeragentmove][3] = self.fields[playeragentmove][3] + 1
              if self.fields[playeragentmove][2] == 1 and playeragentmove >= 5 and playeragentmove <= 12:
                self.fields[playeragentmove][2] = 0
                self.fields[0][2] = self.fields[0][2] + 1
                #print("Агент забрал шашку на" , self.fields[playeragentmove][0])
              #print("Агенту выпал" , agentroll)
              #print("Агент ходит" , playeragentmove)
              return playeragentmove
            else:
              rrr = 0#print("Выпал 0 и агент не делает выбора")
     #while True:
     # thisbotmove = first()
     # if thisbotmove != 4 or thisbotmove != 8 or thisbotmove != 14:
     #    #print("нет второго хода бота")
     #    break

     if self.fields[16][3] != 1:
      thisbotmove = first()
      #print(first())
      if thisbotmove == 4 or thisbotmove == 8 or thisbotmove == 14:
       #print("Второй ход бота", thisbotmove)
       self.fields[16][2] = 1
      else:
       self.fields[16][2] = 0
    # if thisbotmove == 4 or thisbotmove == 8 or thisbotmove == 14:
     if self.fields[16][2] != 1:
      thisagentmove = second()
      #print(second())
      if thisagentmove == 4 or thisagentmove == 8 or thisagentmove == 14:
        #print("Второй ход агента", thisagentmove)
        self.fields[16][3] = 1
      else:
        self.fields[16][3] = 0
     #while True:
     # thisagentmove =  second()
     # if thisagentmove != 4 or thisagentmove != 8 or thisagentmove != 14:
     #    #print("нет второго хода агента")
     #    break


     terminated = bool(self.fields[15][2] >= 7 or self.fields[15][3] >= 7)
     truncated = False  # we do not limit the number of steps here

     reward = 0
     if self.fields[15][3] >= 7 and self.fields[15][2] != 7:
       reward = 1

    # Optionally we can pass additional info, we are not using that for now
     info = {}

     self.fields = self.fields.flatten('C')

     self.fields = [x/16 for x in self.fields]

     #print(self.fields)

     return np.array(self.fields).astype(np.float32), reward, terminated, truncated, info


  def render(self):
        # agent is represented as a cross, rest as a dot
        if self.render_mode == "console":
            1==1

    #print(self.fields[0])

  def close(self):
    pass

In [18]:
import ray

ray.init()

RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

In [ ]:
from ray import tune

# Register , how your env should be constructed (always with 5, or you can take values from the `config` EnvContext object):
tune.register_env("my_env", lambda config: GoLeftEnv())

2023-08-25 22:19:14,916	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
try:
    import gymnasium as gym
    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = "my_env"
env = GoLeftEnv()
#algo = PPOConfig().environment(env_name).rollouts(num_rollout_workers=1).build()
iii = 0
while iii<1000:
  iii = iii+1
  episode_reward = 0
  terminated = truncated = False

  if gymnasium:
      obs, info = env.reset()
  else:
      obs = env.reset()

  while not terminated and not truncated:
      action = env.action_space.sample()
      action = 1
      if gymnasium:
          obs, reward, terminated, truncated, info = env.step(action)
      else:
          obs, reward, terminated, info = env.step(action)
      episode_reward += reward
  print(episode_reward)

In [ ]:
#одна тестовая игра
try:
    import gymnasium as gym
    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = "my_env"
env = GoLeftEnv()
#algo = PPOConfig().environment(env_name).rollouts(num_rollout_workers=1).build()
iii = 0
while iii<1:
  iii = iii+1
  episode_reward = 0
  terminated = truncated = False

  if gymnasium:
      obs, info = env.reset()
  else:
      obs = env.reset()

  while not terminated and not truncated:
      action = env.action_space.sample()
      #action = 1
      #action = algo2.compute_single_action(obs)
      if gymnasium:
          obs, reward, terminated, truncated, info = env.step(action)
      else:
          obs, reward, terminated, info = env.step(action)
      episode_reward += reward
      print(action)
      obs = [x if x == 0 else int(x*16) for x in obs]
      obs = [int(a) for a in obs]
      obs = np.array(obs)
      obs = obs.reshape(17, 4)
      print(obs)
      print(reward)
  print(episode_reward)

In [ ]:
from ray import tune

tune.run("PPO",
         config={"env": "my_env",
                 "evaluation_interval": 10,
                 "evaluation_num_episodes": 50,
                 "num_workers": 2,
                 # other configurations go here, if none provided, then default configurations will be used
                 },
         storage_path="Urrnow70",
         checkpoint_freq=10
         )

2023-08-25 22:19:40,623	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-08-25 22:19:40,624	WARNING syncer.py:260 -- You are using remote storage, but you don't have `fsspec` installed. This can lead to inefficient syncing behavior. To avoid this, install fsspec with `pip install fsspec`. Depending on your remote storage provider, consider installing the respective fsspec-package (see https://github.com/fsspec).
2023-08-25 22:20:07,039	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-08-25 22:20:07,405	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-08-25 22:20:07,409	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` ha

2023-08-25 22:20:07,563	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-25 22:20:07,564	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=2606) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=2606) 2023-08-25 22:20:12,365	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. Wh

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_my_env_8c218_00000,64000,"{'ObsPreprocessorConnector_ms': 0.0048182010650634766, 'StateBufferConnector_ms': 0.0033750534057617188, 'ViewRequirementAgentConnector_ms': 0.26031041145324707}","{'num_env_steps_sampled': 64000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 64000, 'num_agent_steps_trained': 0}",{},68.74,{},1,0.48,0,57,"{'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 0.11531132242497383}, 'default_policy': {'total_loss': 0.11531132242497383, 'policy_loss': -0.004669851620496908, 'vf_loss': 0.11864918437816187, 'vf_loss_unclipped': 0.11864918437816187, 'vf_explained_var': 0.04657289284124557, 'entropy': 1.6665341601188757, 'mean_kl_loss': 0.005919951064206805, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.22500000894069672}}, 'num_env_steps_sampled': 64000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 64000, 'num_agent_steps_trained': 0}",64000,0,64000,4000,319.341,0,0,0,0,2,0,0,0,"{'cpu_util_percent': 38.37222222222222, 'ram_util_percent': 26.927777777777774}",{},{},{},"{'mean_raw_obs_processing_ms': 0.5149210451923741, 'mean_inference_ms': 1.4596046309567856, 'mean_action_processing_ms': 0.11771713486815756, 'mean_env_wait_ms': 0.1416960313486475, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 1.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 0.48, 'episode_len_mean': 68.74, 'episode_media': {}, 'episodes_this_iter': 57, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'episode_lengths': [64, 69, 72, 73, 72, 62, 71, 61, 74, 69, 62, 64, 66, 61, 72, 61, 61, 77, 70, 74, 55, 60, 73, 66, 73, 78, 73, 70, 69, 62, 61, 68, 69, 73, 65, 61, 69, 67, 60, 80, 71, 60, 66, 64, 63, 64, 71, 66, 78, 59, 60, 54, 69, 74, 66, 67, 71, 72, 66, 54, 76, 77, 71, 77, 63, 71, 82, 64, 75, 70, 70, 74, 77, 84, 65, 76, 66, 60, 81, 76, 72, 74, 68, 67, 58, 68, 76, 66, 72, 70, 67, 72, 66, 56, 73, 78, 80, 66, 74, 74]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.5149210451923741, 'mean_inference_ms': 1.4596046309567856, 'mean_action_processing_ms': 0.11771713486815756, 'mean_env_wait_ms': 0.1416960313486475, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.0048182010650634766, 'StateBufferConnector_ms': 0.0033750534057617188, 'ViewRequirementAgentConnector_ms': 0.26031041145324707}}","{'training_iteration_time_ms': 13371.129, 'sample_time_ms': 4593.727, 'synch_weights_time_ms': 4.259}"


2023-08-25 22:24:01,380	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-08-25 22:24:11,427	INFO tune.py:1148 -- Total run time: 270.80 seconds (233.82 seconds for the tuning loop).
2023-08-25 22:24:11,431	WARNING tune.py:1163 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)


In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm

algo2 = Algorithm.from_checkpoint("/workspaces/codespaces-jupyter/notebooks/Urrnow70/PPO/PPO_my_env_8c218_00000_0_2023-08-25_22-20-07/checkpoint_000010")

2023-08-25 22:24:20,812	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-25 22:24:20,814	WARNING algorithm_config.py:656 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-08-25 22:24:20,816	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.e

In [ ]:
try:
    import gymnasium as gym
    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = "my_env"
env = GoLeftEnv()
#algo = PPOConfig().environment(env_name).rollouts(num_rollout_workers=1).build()
iii = 0
while iii<100:
  iii = iii+1
  episode_reward = 0
  terminated = truncated = False

  if gymnasium:
      obs, info = env.reset()
  else:
      obs = env.reset()

  while not terminated and not truncated:
      action = algo2.compute_single_action(obs)
      if gymnasium:
          obs, reward, terminated, truncated, info = env.step(action)
      else:
          obs, reward, terminated, info = env.step(action)
      episode_reward += reward
  print(episode_reward)

In [ ]:
import numpy as np
#одна тестовая игра
try:
    import gymnasium as gym
    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = "my_env"
env = GoLeftEnv()
#algo = PPOConfig().environment(env_name).rollouts(num_rollout_workers=1).build()
iii = 0
while iii<1:
  iii = iii+1
  episode_reward = 0
  terminated = truncated = False

  if gymnasium:
      obs, info = env.reset()
  else:
      obs = env.reset()

  while not terminated and not truncated:
      action = algo2.compute_single_action(obs)
      #action = 1
      #action = algo2.compute_single_action(obs)
      if gymnasium:
          obs, reward, terminated, truncated, info = env.step(action)
      else:
          obs, reward, terminated, info = env.step(action)
      episode_reward += reward
      print(action)
      obs = [x if x == 0 else int(x*16) for x in obs]
      obs = [int(a) for a in obs]
      obs = np.array(obs)
      obs = obs.reshape(17, 4)
      print(obs)
      print(reward)
      obs = obs.flatten('C')
      sobs = [x/16 for x in obs]
  print(episode_reward)

In [ ]:
import numpy as np
#одна тестовая игра с рендерингом
try:
    import gymnasium as gym
    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = "my_env"
env = GoLeftEnv()
#algo = PPOConfig().environment(env_name).rollouts(num_rollout_workers=1).build()
iii = 0
while iii<1:
  iii = iii+1
  episode_reward = 0
  terminated = truncated = False

  if gymnasium:
      obs, info = env.reset()
  else:
      obs = env.reset()

  while not terminated and not truncated:
      action = algo2.compute_single_action(obs)
      #action = 1
      #action = algo2.compute_single_action(obs)
      if gymnasium:
          obs, reward, terminated, truncated, info = env.step(action)
      else:
          obs, reward, terminated, info = env.step(action)
      episode_reward += reward
      print(action)
      obs = [x if x == 0 else int(x*16) for x in obs]
      obs = [int(a) for a in obs]
      obs = np.array(obs)
      obs = obs.reshape(17, 4)
      print(obs)
      print(reward)
      if obs[4][2] == 1:
       print(u"\U0001F7E5", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[5][2] == 1:
       print(u"\U0001F7E5", end="")
      elif obs[5][3] == 1:
       print(u"\U0001F7E6", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[4][3] == 1:
       print(u"\U0001F7E6")
      else:
       print(u"\u2B1C")
      if obs[3][2] == 1:
       print(u"\U0001F7E5", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[6][2] == 1:
       print(u"\U0001F7E5", end="")
      elif obs[6][3] == 1:
       print(u"\U0001F7E6", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[3][3] == 1:
       print(u"\U0001F7E6")
      else:
       print(u"\u2B1C")
      if obs[2][2] == 1:
       print(u"\U0001F7E5", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[7][2] == 1:
       print(u"\U0001F7E5", end="")
      elif obs[7][3] == 1:
       print(u"\U0001F7E6", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[2][3] == 1:
       print(u"\U0001F7E6")
      else:
       print(u"\u2B1C")
      if obs[1][2] == 1:
       print(u"\U0001F7E5", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[8][2] == 1:
       print(u"\U0001F7E5", end="")
      elif obs[8][3] == 1:
       print(u"\U0001F7E6", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[1][3] == 1:
       print(u"\U0001F7E6")
      else:
       print(u"\u2B1C")
      print(u"\u2B1B", end="")
      if obs[9][2] == 1:
       print(u"\U0001F7E5", end="")
      elif obs[9][3] == 1:
       print(u"\U0001F7E6", end="")
      else:
       print(u"\u2B1C", end="")
      print(u"\u2B1B")
      print(u"\u2B1B", end="")
      if obs[10][2] == 1:
       print(u"\U0001F7E5", end="")
      elif obs[10][3] == 1:
       print(u"\U0001F7E6", end="")
      else:
       print(u"\u2B1C", end="")
      print(u"\u2B1B")
      if obs[14][2] == 1:
       print(u"\U0001F7E5", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[11][2] == 1:
       print(u"\U0001F7E5", end="")
      elif obs[11][3] == 1:
       print(u"\U0001F7E6", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[14][3] == 1:
       print(u"\U0001F7E6")
      else:
       print(u"\u2B1C")
      if obs[13][2] == 1:
       print(u"\U0001F7E5", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[12][2] == 1:
       print(u"\U0001F7E5", end="")
      elif obs[12][3] == 1:
       print(u"\U0001F7E6", end="")
      else:
       print(u"\u2B1C", end="")
      if obs[13][3] == 1:
       print(u"\U0001F7E6")
      else:
       print(u"\u2B1C")  
      obs = obs.flatten('C')
      sobs = [x/16 for x in obs]
  print(episode_reward)